In [1]:
from matplotlib import pyplot as plt
from math import floor, ceil
import numpy as np
import cv2 as C

CANNY_THRESH = 400
RADIUS_RANGE = (1, 20)
SCAN_STEP = 8
REGEN_TOL = 0.1
VIDEO_FILE = './data/D.mp4'

# just wrapper to clean up data after hough
hough = lambda img, **kwargs: np.uint16(np.around(C.HoughCircles(img, method=C.HOUGH_GRADIENT, dp=1, **kwargs)))

In [ ]:
vidcap = C.VideoCapture(VIDEO_FILE)
success, img = vidcap.read()
count = 0
if not success:
  print("Failed to read video")
  exit(1)

plt.imshow(img, cmap='gray')
plt.show()

In [ ]:
# img = C.imread(f'./test/001.png', C.IMREAD_GRAYSCALE)
img_orig = img.copy()
edges = C.Canny(img, CANNY_THRESH, CANNY_THRESH)

try:
  detected =  hough(edges,
    minDist=8, param1 = 100, param2 = 15,
    minRadius = RADIUS_RANGE[0], maxRadius = RADIUS_RANGE[1]
  )
except Exception as e:
  edges = C.Canny(img, CANNY_THRESH/2, CANNY_THRESH/2)
  detected =  hough(edges,
    minDist=8, param1 = 100, param2 = 15,
    minRadius = RADIUS_RANGE[0], maxRadius = RADIUS_RANGE[1]
  )

# Find which radius wins, and allow for 10% error
unique, counts = np.unique(detected[0, :, 2], return_counts=True)
counts = dict(zip(unique, counts))
R = max(counts, key=counts.get)
E = round(REGEN_TOL * R)

dfilt = detected[0, np.where((detected[0, :, 2] >= R-E) & (detected[0, :, 2] <= R+E))]
R = np.mean(dfilt[0, :, 2])

rpe, rme = ceil(R+E), floor(R-E)
print(f"Using R={R:.2f} (seen {max(counts)} times) with ϵ=±{E} => R in ({rme:.2f}, {rpe:.2f})")

In [ ]:
# 2nd round of clean Hough
detected = hough(edges,
  minDist=img.shape[0] / (2 * (R + E)), param1=100, param2=10,
  minRadius=rme, maxRadius=rpe
)

# Binning 'y's into boxes of size R
detected[0, :, 1] = np.round(detected[0, :, 1] / R) * R

# count 'y's
unique, counts = np.unique(detected[0, :, 2], return_counts=True)
counts = dict(zip(unique, counts))

# choose only the mode 'y' and 'R'
max_R = max(counts, key=counts.get)
max_Y = max(detected[0, :, 1])
detected = detected[:, (detected[0, :, 2] == max_R) & (detected[0, :, 1] == max_Y), :]

# sort 'x's
detected = detected[:, detected[0, :, 0].argsort(), :]
diffs = np.round(np.diff(detected[0, :, 0]) / max_R, 1) - 2

# finding mean diff by counting the most common diff
unique, counts = np.unique(diffs, return_counts=True)
counts = dict(zip(unique, counts))
max_diff = max(counts, key=counts.get)

print(f"Mean radius: {max_R}px, Mean distance: {max_diff:.1f}R (y-axis={max_Y})")

# Draw
for pt in detected[0, :]:
  C.circle(img, (pt[0], pt[1]), pt[2], (255, 0, 0), 2)
plt.imshow(img, cmap='grey')
plt.axis('off')
plt.show()

In [5]:
data_dict = {
  'R': str(max_R),
  'Y': str(max_Y),
  'D': str(max_diff)
}

with open('./data.txt', 'w') as f:
  from json import dumps
  data = dumps(data_dict)
  f.write(str(data))

In [6]:
!rm -rf ./in_dir/* ./out_dir/*

In [ ]:
!ffmpeg -hide_banner -loglevel error -i "{VIDEO_FILE}" './in_dir/%03d.png'
!python3 src/extract.py in_dir out_dir {max_Y}
!rm -r ./out_dir/0.png ./out_dir/000.png